In [8]:
from syllablecounter import load_model, get_data
import numpy as np

In [2]:
def printmd(string, size = 3):
    from IPython.display import Markdown, display
    display(Markdown(f'<font size = {size}>{string}</font>'))
def green(string, deg = 1):
    return f'<span style = "background-color:hsl(125, {deg * 100}%, 50%);">{string}</span>'
def red(string, deg = 1):
    return f'<span style = "background-color:hsl(0, {deg * 100}%, 50%);">{string}</span>'
def ctext(string, deg = 0.5):
    deg = 2 * deg - 1
    if deg > 0:
        return green(string, deg)
    else:
        return red(string, -deg)

In [3]:
counter = load_model()
counter

SyllableCounter(
  (embed): Embedding(30, 67)
  (rnn): GRU(67, 134, num_layers=3, dropout=0.06533740023319776, bidirectional=True)
  (drops): ModuleList(
    (0): Dropout(p=0.32429407475156746, inplace=False)
  )
  (norms): ModuleList(
    (0): BatchNorm1d(268, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (lins): ModuleList(
    (0): Linear(in_features=268, out_features=268, bias=True)
  )
  (out): Linear(in_features=268, out_features=1, bias=True)
)

In [4]:
import pandas as pd
df = pd.read_csv('data/gutsyls.tsv', sep = '\t')
df.head()

,words,syl_seq,syls
0,nondistillation,100100100101000,5
1,transitoriness,10001010101000,5
2,bipyramidal,10101001100,5
3,nonciteable,10010001100,4
4,livebearer,1000100010,3


In [9]:
hard_words = {'words': [], 'syl_seq': [], 'syls': []}
for idx in np.random.choice(range(len(df)), size = 1000):
    yhat = counter.predict(df['words'][idx], return_confidence = True, return_sequence = True)
    if yhat['num_syls'] != df['syls'][idx]:
        hard_words['words'].append(df['words'][idx])
        hard_words['syl_seq'].append(df['syl_seq'][idx])
        hard_words['syls'].append(df['syls'][idx])
        print(df['words'][idx], df['syl_seq'][idx], df['syls'][idx], yhat['num_syls'], yhat['confidence'])
        print(yhat['probs'])

dessiatine 1001001000 3 4 0.29
[('d', 0.88), ('e', 0.11), ('s', 0.12), ('s', 0.88), ('i', 0.12), ('a', 0.89), ('t', 0.88), ('i', 0.12), ('n', 0.11), ('e', 0.11)]
hielamon 10010100 3 4 0.19
[('h', 0.88), ('i', 0.12), ('e', 0.62), ('l', 0.25), ('a', 0.76), ('m', 0.88), ('o', 0.12), ('n', 0.11)]
caramuel 10100100 3 4 0.08
[('c', 0.88), ('a', 0.12), ('r', 0.48), ('a', 0.52), ('m', 0.72), ('u', 0.29), ('e', 0.87), ('l', 0.11)]
goutt 10001 2 1 0.55
[('g', 0.88), ('o', 0.11), ('u', 0.11), ('t', 0.11), ('t', 0.11)]
enneagon 10101100 4 3 0.31
[('e', 0.89), ('n', 0.12), ('n', 0.88), ('e', 0.12), ('a', 0.12), ('g', 0.81), ('o', 0.18), ('n', 0.11)]
argoan 101010 3 2 0.47
[('a', 0.88), ('r', 0.12), ('g', 0.88), ('o', 0.12), ('a', 0.12), ('n', 0.11)]
feuille 1000010 2 1 0.42
[('f', 0.88), ('e', 0.11), ('u', 0.11), ('i', 0.12), ('l', 0.11), ('l', 0.13), ('e', 0.11)]
oceanology 1100010110 5 6 0.21
[('o', 0.89), ('c', 0.88), ('e', 0.12), ('a', 0.89), ('n', 0.75), ('o', 0.25), ('l', 0.12), ('o', 0.88), 

In [6]:
while True:
    words = input()
    outputs = counter.predict(words, return_sequence = True, pred_threshold = 0.5)
    string = ''.join([ctext(f'{char}', deg) for char, deg in outputs['probs']])
    printmd(f'{string}&nbsp;= {outputs["num_syls"]} syllables', size = 5)

KeyboardInterrupt: 